In [2]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
def WriteBandEdgeZnGeN2():
    #BandEdges = !grep -A 1 "52        " ~/remote/guild/work/ZnGeN2/Zn_w_d_Ge_no_d_N/HSE25/Gpoint/EIGENVAL
    BandEdge = {}

    CBand = !grep " 53      " ~/remote/cori/work/copyscratch/ZnSiN2/bandstruct/GX1/EIGENVAL
    BandEdge['ZnSiN2_CB'] = float(CBand[8].split()[1])
    VBand = !grep " 52      " ~/remote/cori/work/copyscratch/ZnSiN2/bandstruct/ZU6/EIGENVAL
    BandEdge['ZnSiN2_VB'] = float(VBand[17].split()[1])
    
    BandEdges = !grep -A 1 "52        " ~/remote/cori/work/ZnGeN2prim/ZnGeN2_HSE25_bandstruct/GZ2/EIGENVAL
    BandEdge['ZnGeN2_VB'] = float(BandEdges[24].split()[1])
    BandEdge['ZnGeN2_CB'] = float(BandEdges[25].split()[1])
    
    BandEdges = !grep -A 1 "52        " ~/remote/cori/work/copyscratch/ZnSnN2/bandstruct/GX1/EIGENVAL
    BandEdge['ZnSnN2_VB'] = float(BandEdges[24].split()[1])
    BandEdge['ZnSnN2_CB'] = float(BandEdges[25].split()[1])
    
    BandEdges = !head -17 ~/remote/cori/work/polar/HSE/GaN/prim/25/EIGENVAL | tail -2
    BandEdge['GaN25_VB'] = float(BandEdges[0].split()[1])
    BandEdge['GaN25_CB'] = float(BandEdges[1].split()[1])

    BandEdges = !head -17 ~/remote/cori/work/polar/HSE/GaN/prim/295/EIGENVAL | tail -2
    BandEdge['GaN295_VB'] = float(BandEdges[0].split()[1])
    BandEdge['GaN295_CB'] = float(BandEdges[1].split()[1])

    
    return BandEdge

In [12]:
WriteBandEdgeZnGeN2()

{'GaN25_CB': 5.245438,
 'GaN25_VB': 2.062347,
 'GaN295_CB': 5.447876,
 'GaN295_VB': 1.965016,
 'ZnGeN2_CB': 5.98107,
 'ZnGeN2_VB': 2.791913,
 'ZnSiN2_CB': 8.840683,
 'ZnSiN2_VB': 4.009651,
 'ZnSnN2_CB': 4.613189,
 'ZnSnN2_VB': 3.208647}

In [4]:
def WriteDefects():
    mainF = "/home/na/remote/cori/work/copyscratch/ZnGeN2/"
    listDef = !ls "{mainF##*/}"
    chargeDict = {'n4':'-4','n3':'-3','n2':'-2','n1':'-1','neutral':'0','p1':'1','p2':'2','p3':'3','p4':'4',}
    E = {}
    for defect in listDef:
        chargeF = mainF + defect
        listcharge = !ls "{chargeF##*/}"
        Edef = {}
        for charge in listcharge:
            OUT = chargeF + '/' + charge +"/OUTCAR"
            loc = !grep "energy  w" "{OUT##*/}"
            if charge in chargeDict:
                Edef[chargeDict[charge]] = float(loc[0].split()[-1])
            elif charge == 'H1':
                HchargeF = mainF + defect +'/'+ charge
                listHcharge = !ls "{HchargeF##*/}"
                HEdef = {}
                POS = chargeF + '/' + charge + '/' + listHcharge[0] + '/POSCAR'
                POSloc = !head -7 "{POS##*/}" | tail -2
                for Hcharge in listHcharge:
                    OUTH = chargeF + '/' + charge + '/' + Hcharge +"/OUTCAR"
                    loc = !grep "energy  w" "{OUTH##*/}"
                    HEdef[chargeDict[Hcharge]] = float(loc[0].split()[-1])
                E[defect+'-H'] = [HEdef, {POSloc[0].split()[i]:map(int, POSloc[1].split())[i] for i in range(len(POSloc[0].split()))}]
        if not Edef:
            continue
        POS = chargeF + '/neutral/POSCAR'
        POSloc = !head -7 "{POS##*/}" | tail -2
        E[defect] = [Edef, {POSloc[0].split()[i]:map(int, POSloc[1].split())[i] for i in range(len(POSloc[0].split()))}]
    return E

In [18]:
def WriteDefectsSi():
    mainF = "/home/nadamski/remote/cori/work/copyscratch/ZnSiN2/defects/"
    listDef = !ls "{mainF##*/}"
    chargeDict = {'n4':'-4','n3':'-3','n2':'-2','n1':'-1','neutral':'0','p1':'1','p2':'2','p3':'3','p4':'4',}
    E = {}
    for defect in listDef:
        print defect
        chargeF = mainF + defect
        listcharge = !ls "{chargeF##*/}"
        Edef = {}
        for charge in listcharge:
            OUT = chargeF + '/' + charge +"/OUTCAR"
            loc = !grep "energy  w" "{OUT##*/}"
            if charge in chargeDict:
                Edef[chargeDict[charge]] = float(loc[0].split()[-1])
            elif charge == 'H1':
                HchargeF = mainF + defect +'/'+ charge
                listHcharge = !ls "{HchargeF##*/}"
                HEdef = {}
                POS = chargeF + '/' + charge + '/' + listHcharge[0] + '/POSCAR'
                POSloc = !head -7 "{POS##*/}" | tail -2
                for Hcharge in listHcharge:
                    OUTH = chargeF + '/' + charge + '/' + Hcharge +"/OUTCAR"
                    loc = !grep "energy  w" "{OUTH##*/}"
                    HEdef[chargeDict[Hcharge]] = float(loc[0].split()[-1])
                E[defect+'-H'] = [HEdef, {POSloc[0].split()[i]:map(int, POSloc[1].split())[i] for i in range(len(POSloc[0].split()))}]
        if not Edef:
            continue
        print Edef
        POS = chargeF + '/' + listcharge[0] + '/POSCAR'
        POSloc = !head -7 "{POS##*/}" | tail -2
        print POSloc
        E[defect] = [Edef, {POSloc[0].split()[i]:map(int, POSloc[1].split())[i] for i in range(len(POSloc[0].split()))}]
    return E

In [6]:
def WriteDefectsScN():
    mainF = "/home/na/remote/cori/work/copyscratch/ScN/ScN_rs/ScN_222/"
    listDef = !ls "{mainF##*/}"
    chargeDict = {'n4':'-4','n3':'-3','n2':'-2','n1':'-1','neutral':'0','p1':'1','p2':'2','p3':'3','p4':'4',}
    E = {}
    for defect in listDef:
        print defect
        chargeF = mainF + defect
        listcharge = !ls "{chargeF##*/}"
        Edef = {}
        for charge in listcharge:
            OUT = chargeF + '/' + charge +"/OUTCAR"
            loc = !grep "energy  w" "{OUT##*/}"
            if charge in chargeDict:
                Edef[chargeDict[charge]] = float(loc[0].split()[-1])
            elif charge == 'H1':
                HchargeF = mainF + defect +'/'+ charge
                listHcharge = !ls "{HchargeF##*/}"
                HEdef = {}
                POS = chargeF + '/' + charge + '/' + listHcharge[0] + '/POSCAR'
                POSloc = !head -7 "{POS##*/}" | tail -2
                for Hcharge in listHcharge:
                    OUTH = chargeF + '/' + charge + '/' + Hcharge +"/OUTCAR"
                    loc = !grep "energy  w" "{OUTH##*/}"
                    HEdef[chargeDict[Hcharge]] = float(loc[0].split()[-1])
                E[defect+'-H'] = [HEdef, {POSloc[0].split()[i]:map(int, POSloc[1].split())[i] for i in range(len(POSloc[0].split()))}]
        if not Edef:
            continue
        POS = chargeF + '/neutral/POSCAR'
        POSloc = !head -7 "{POS##*/}" | tail -2
        E[defect] = [Edef, {POSloc[0].split()[i]:map(int, POSloc[1].split())[i] for i in range(len(POSloc[0].split()))}]
    return E

In [14]:
def WriteChemPot():
    mainF = "/home/nadamski/remote/cori/work/copyscratch/refchem"
    listChem = !ls "{mainF##*/}"
    E = {}
    for chem in listChem:
        print chem
        OUT = mainF +'/'+ chem + '/OUTCAR'
        POS = mainF +'/'+ chem + '/POSCAR'
        loc = !grep "energy  w" "{OUT##*/}"
        POSloc = !head -7 "{POS##*/}" | tail -2
        E[chem] = [float(loc[-1].split()[-1]), {POSloc[0].split()[i]:map(int, POSloc[1].split())[i] for i in range(len(POSloc[0].split()))}]
    return E

In [8]:
Corrections = {'default':{},
               'V_N':{'-3':1.49304, '-2':0.676906, '-1':0.214226, '1':0.159226, '2':0.596906, '3':1.34304},
               'V_Ge':{'-4':3.38762, '-3':1.94304, '-2':0.906906, '-1':0.269226, '1':0.196226, '2':0.596906, '3':1.34304},
               'V_Zn':{'-2':0.916906, '-1':0.259226, '1':0.103226, '2':0.606906},
               'Ge_Zn':{'-3':1.27704, '-2':0.466906, '-1':0.0992264, '1':0.249226, '2':0.886906, '3':1.64304, '4':2.74762},
               'Zn_Ge':{'-3':1.73904, '-2':0.876906, '-1':0.239226, '1':0.129226, '2':0.796906},
               'ZnGePair':{'-1':0.202226, '1':0.189226},
               'V_Zn-H':{'-2':0.810906, '-1':0.202226, '1':0.149226},
               'V_N-H':{'-1':239226, '1':0.139226, '2':0.596906, '3':1.25304},
               'Ge_N':{'-2':0.636906, '-1':0.234226, '1':0.124226, '2':0.706906},
               'N_Ge':{'-2':0.802906, '-1':0.269226, '1':0.129226, '2':0.596906},
               'Zn_N':{'-1':0.239226, '1':0.0992264},
               'N_Zn':{'-1':0.259226, '1':0.0792264},
               'C_Ge':{'-1':0.189226, '1':0.0992264},
               'C_N':{'-2':0.666906, '-1':0.159226, '1':0.199026},
               'O_N':{'-1':0.114226, '1':0.185226, '2':0.636906},
               'H_i':{'-1':0.189226, '1':0.149226},
               'N_i':{'-2':0.716906, '-1':0.209226, '1':0.134226, '2':0.596906, '3':1.40304},
               'Zn_i':{'1':0.179226, '2':0.656906},
               'Ge_i':{'1':0.179226, '2':0.770906, '3':1.13304, '4':2.26762},
               'Li_Zn':{'-1':0.219226},
               'Li_Zn-H':{'-1':0.149226,'1':0.119226},
               'Li_i':{'-1':0.0992264,'1':0.189226},
               'LiPair':{'-1':0.119226, '1':0.149226},
               'Li_Ge':{'-1':0.279226, '-2':0.956906, '-3':1.79304},
               'Cu_Zn':{'-1':0.171226, '1':0.219226},
               'Al_Ge':{'-1':0.194226},
               'Al_Zn':{'1':0.249226},
               'AlPair':{'-1':0.0442264,'1':0.239226},
               'Al_i':{'1':0.259226, '2':0.916906, '3':1.91304},
               'Ga_d_Ge':{'-1':0.204226},
               'Ga_Ge':{'-1':0.239226},
               'Ga_d_Zn':{'1':0.243226},
               'Ga_Zn':{'1':0.209226},
               'GaPair':{'-1':0.134226,'1':0.119226},
               'Ga_i':{'1':0.239226, '2':0.836906, '3':1.78104},
               'In_Ge':{'-1':0.174226},
               'In_Zn':{'1':0.279226},
               'InPair':{},
               'Ga_Ge-H':{'-1':0.191226, '1':0.0692264},
               'Al_Ge-H':{'-1':0.204226, '1':0.189226},
               'Sn_Zn':{'2':0.936906},
               'Sn_Ge':{},
               'P_Ge':{'-1':0.0692264, '1':0.216226},
               'P_N':{'-1':0.119226, '1':0.199226,'2':0.756906},
               'P_Zn':{'-1':0.119226, '1':0.219226, '2':0.916906, '3':1.91304},
               'As_Ge':{'-1':0.059226, '1':0.219226},
               'As_N':{'-1':0.134226, '1':0.179226,'2':0.736906},
               'As_Zn':{'-1':0.129226, '1':0.244226, '2':0.896906, '3':1.91304},
               'S_N':{'-1':0.0642264, '1':0.212226},
               'S_Ge':{'-1':0.209226, '1':0.196226, '2':0.776906},
               'S_Zn':{'-1':0.0292264, '1':0.269226, '2':0.876906, '3':1.82304, '4':3.26762},
               'Se_N':{'-1':0.0842264, '1':0.199226},
               'Se_Ge':{'-1':0.124226, '1':0.214226, '2':0.856906},
               'Se_Zn':{'-1':0.129226, '1':0.289226, '2':0.876906, '3':1.91304, '4':3.34762}}
np.save('FreysoldtCorrections.npy', Corrections)

In [23]:
CorrectionsSi = {'default':{},
                 'H_i':{'-1':0.20862, '1':0.18062},
                 'Zn_Si':{'-2':1.03448, '-1':0.31862, '1':0.0686203},
                 'V_Zn':{'-2':0.974481, '-1':0.27362, '1':0.0886203},
                 'V_Si':{'-4':3.97793, '-3':2.34258, '-2':1.19448, '-1':0.39862},
                 'S_N':{'-1':0.16862 ,'1':0.24862},
                 'O_N':{'-1':0.23862 ,'1':0.18762},
                 'P_Si':{'-1':0.0886203, '1':0.20862},
                 'As_Si':{'-1':0.21862, '1':0.19862},
                 'Se_N':{'-1':0.17862, '1':0.19862},
                 'Al_Si':{'-1':0.23862},
                 'Al_Zn':{'1':0.26862},
                 'Ga_Si':{'-1':0.28862},
                 'Ga_Zn':{'1':0.21362},
                 'In_Si':{'-1':0.21362},
                 'In_Zn':{'1':0.27862},
                 'P_N':{'1':0.19862, '2':0.714481},
                 'As_N':{'1':0.18862}
                 'S_Zn':{'1':0.26862}
                 'S_Ge':{'1':0.17862}
}
np.save('FreysoldtCorrectionsSi.npy', CorrectionsSi)

In [19]:
CorrectionsScN = {'default':{},
                  'H_i':{'-1':0.144162, '1':0.194162}}
np.save('FreysoldtCorrectionsScN.npy', CorrectionsScN)
#Artificial dielectric constant 8.8

In [20]:
def writeDielectric():
    Dielec = {}
    Tensor = !grep -A 4 "DIELECTRIC TENSOR" ~/remote/cori/work/polar/HSE/ZnSiN2/dielec/current/OUTCAR
    MicTensor = np.array([(Tensor[2].split()),(Tensor[3].split()),(Tensor[4].split())]).astype(np.float)
    MacTensor = np.array([(Tensor[8].split()),(Tensor[9].split()),(Tensor[10].split())]).astype(np.float)
    
    Dielec['ZnSiN2'] = MicTensor+MacTensor # 8.8
    
    Tensor = !grep -A 4 "DIELECTRIC TENSOR" ~/remote/cori/work/polar/HSE/ZnGeN2/dielec/current/OUTCAR
    MicTensor = np.array([(Tensor[2].split()),(Tensor[3].split()),(Tensor[4].split())]).astype(np.float)
    MacTensor = np.array([(Tensor[8].split()),(Tensor[9].split()),(Tensor[10].split())]).astype(np.float)
    
    Dielec['ZnGeN2'] = MicTensor+MacTensor
    return Dielec
writeDielectric()

{'ZnGeN2': array([[ 9.93 ,  0.   ,  0.   ],
        [ 0.   ,  9.78 ,  0.   ],
        [ 0.   ,  0.   ,  9.735]]), 'ZnSiN2': array([[ 8.947,  0.   ,  0.   ],
        [ 0.   ,  8.68 ,  0.   ],
        [ 0.   ,  0.   ,  8.869]])}

In [3]:
colors = {#'V_Zn':'#ff7f0e',
          'V_Zn':'#dd8800',
          'V_Ge':'#dd33dd',
          'V_N':'#11aaff',
          'V_Si':'#dd0044',
          #'Zn_Ge':'#1f77b4',
          'Zn_Ge':'#bb0077',
          'Zn_Si':'C0',
          'Ge_Zn':'#2ca02c',
          'H_i':'purple',
          'O_N':'#000000',
          'C_N':'#997733',
          'C_Ge':'#995555',
          'S_N':'#00bbbb',
          'S_Zn':'#666644',
          'S_Ge':'#9999dd',
          'Se_N':'#00bb22',
          'Se_Zn':'#666644',
          'Se_Ge':'#9999dd',
          'P_Ge':'#0000ff',
          'P_Si':'#0000ff',
          'P_Zn':'#226666',
          'P_N':'#886666',
          'As_Ge':'#6600ff',
          'As_Zn':'#226666',
          'As_N':'#aa6666',
          'V_Zn-H':'#cf4f0e',
          'Al_Ge':'#ff7030',
          'Al_Ge-H':'#ff2020',
          'Al_Zn':'#ff0070',
          'AlPair':'C5',
          'Al_i':'C2',
          'Ga_Ge':'#ff7030',
          'Ga_Ge-H':'#ff2020',
          'Ga_Zn':'#ff0070',
          'GaPair':'C5',
          'Ga_i':'C2',
          'In_Zn':'#ff0066',
          'In_Ge':'#ff7030',
          'InPair':'C5',
          'Cu_Zn':'C3',
          'Li_Zn':'C1',
          'Li_Zn-H':'#ff2020',
          'Li_i':'C2',
          'LiPair':'C5',
          'Li_Ge':'#ff7030'}

np.save('LineColors.npy', colors)


In [20]:
text_def = {'V_Zn':r'$V_{\rm Zn}$',
            'V_Ge':r'$V_{\rm Ge}$',
            'V_N':r'$V_{\rm N}$',
            'V_Si':r'$V_{\rm Si}$',
            'Ge_Zn':r'$\rm Ge_{Zn}$',
            'Zn_Ge':r'$\rm Zn_{Ge}$',
            'Zn_Si':r'$\rm Zn_{Si}$',
            'O_N':r'$\rm O_N$',
            'S_N':r'$\rm S_N$',
            'S_Ge':r'$\rm S_{Ge}$',
            'S_Zn':r'$\rm S_{Zn}$',
            'Se_N':r'$\rm Se_N$',
            'Se_Ge':r'$\rm Se_{Ge}$',
            'Se_Zn':r'$\rm Se_{Zn}$',
            'H_i':r'H$_i$',
            'Se_N':r'$\rm Se_N$',
            'C_N':r'$\rm C_N$',
            'C_Ge':r'$\rm C_{Ge}$',
            'Al_Ge':r'$\rm Al_{Ge}$',
            'Al_Ge-H':r'$\rm Al_{Ge}-H$',
            'Al_Zn':r'$\rm Al_{Zn}$',
            'AlPair':r'$\rm Al_{Zn}-Al_{Ge}$',
            'Al_i':r'${\rm Al}_{i}$',
            'Ga_Ge':r'$\rm Ga_{Ge}$',
            'Ga_Ge-H':r'$\rm Ga_{Ge}-H$',
            'Ga_Zn':r'$\rm Ga_{Zn}$',
            'GaPair':r'$\rm Ga_{Zn}-Ga_{Ge}$',
            'Ga_i':r'${\rm Ga}_{i}$',
            'V_Zn-H':r'$\rm V_{Zn}-H$',
            'In_Zn':r'$\rm In_{Zn}$',
            'In_Ge':r'$\rm In_{Ge}$',
            'InPair':r'$\rm In_{Zn}-In_{Ge}$',
            'P_Zn':r'$\rm P_{Zn}$',
            'P_N':r'$\rm P_{N}$',
            'P_Ge':r'$\rm P_{Ge}$',
            'P_Si':r'$\rm P_{Si}$',
            'As_Zn':r'$\rm As_{Zn}$',
            'As_N':r'$\rm As_{N}$',
            'As_Ge':r'$\rm As_{Ge}$',
            'As_Si':r'$\rm As_{Si}$',
            'Al_Si':r'$\rm Al_{Si}$',
            'Ga_Si':r'$\rm Ga_{Si}$',
            'In_Si':r'$\rm In_{Si}$',
            'Cu_Zn':r'$\rm Cu_{Zn}$',
            'Li_Zn':r'$\rm Li_{Zn}$',
            'Li_Zn-H':r'$\rm Li_{Zn}-H$',
            'Li_i':r'Li$_i$', 
            'LiPair':r'${\rm Li_{Zn}}-{\rm Li}_{i}$',
            'Li_Ge':r'$\rm Li_{Ge}$'}


np.save('LineLabels.npy', text_def)


In [23]:
BandEdges = WriteBandEdgeZnGeN2()
np.save('BandEdge.npy', BandEdges)

In [24]:
DefectTotalEnergies = WriteDefects()
np.save('DefectTotalEnergy.npy', DefectTotalEnergies)

In [22]:
DefectTotalEnergiesSi = WriteDefectsSi()
np.save('DefectTotalEnergySi.npy', DefectTotalEnergiesSi)

Al_Si
{'0': -1010.17433212, '-1': -1005.96199062}
['   Zn   Al   Si   N ', '    32     1    31    64']
Al_Zn
{'1': -1026.50733992, '0': -1017.09925428}
['   Zn   Al   Si   N ', '    31     1    32    64']
As_Si
{'1': -1016.82544292, '0': -1008.18954466, '-1': -1000.15428796}
['   Zn   Si   As   N ', '    32    31     1    64']
default
{'0': -1015.05278268}
['Zn Si N', '32 32 64']
Ga_Si
{'0': -1007.27315292, '-1': -1003.04790183}
['   Zn   Ga   Si   N ', '    32     1    31    64']
Ga_Zn
{'1': -1023.92659185, '0': -1014.81433905}
['   Zn   Ga   Si   N ', '    31     1    32    64']
H_i
{'1': -1024.23314707, '0': -1015.84945981, '-1': -1009.23489015}
['   H    Zn   Si   N ', '     1    32    32    64']
h_i_old
In_Si
{'0': -1004.48174738, '-1': -999.94815661}
['   Zn   In   Si   N ', '    32     1    31    64']
In_Zn
{'1': -1022.05778115, '0': -1013.25309724}
['   Zn   In   Si   N ', '    31     1    32    64']
O_N
{'1': -1020.44990164, '0': -1011.54876439, '-1': -1003.0885476}
['   Zn   

In [13]:
DefectTotalEnergiesScN = WriteDefectsScN()
np.save('DefectTotalEnergyScN.npy', DefectTotalEnergiesScN)

default
H_i


In [26]:
ChemicalPotentialReferences= WriteChemPot()
np.save('ChemicalReferences.npy', ChemicalPotentialReferences)

Al
AlN
As
C
Ca
CaN
Cu
Ga
GaN
Ge
Ge3N4
GeAs
GeO2
GeS2
H
In
InN
Li
Li3N
LiN3
Mg
N
O
P
P3N5
S
Sc
ScN
Se
Si
Si3N4
SiO2
Sn
Zn
Zn3As2
Zn3N2
Zn3P2
ZnGeAs2
ZnGeN2
ZnGeP2
ZnO
ZnS
ZnSe
ZnSiN2


In [17]:
bEdges = np.load('BandEdge.npy').item()
DefTot = np.load('DefectTotalEnergy.npy').item()
ChemPot = np.load('ChemicalReferences.npy').item()
FreyCorr = np.load('FreysoldtCorrections.npy').item()

In [11]:
DefTot['In_Zn']

[{'0': -900.23868322, '1': -906.98142526},
 {'Ge': 32, 'In': 1, 'N': 64, 'Zn': 31}]

In [12]:
ChemPot['InN']

[-25.98035876, {'In': 2, 'N': 2}]

In [17]:
BandEdges = WriteBandEdgeZnGeN2()


In [18]:
DefTot

{'AlPair': [{'-1': -900.76799367, '0': -907.56652313},
  {'Al': 2, 'Ge': 31, 'N': 64, 'Zn': 31}],
 'Al_Ge': [{'-1': -897.3254069, '0': -900.16366061},
  {'Al': 1, 'Ge': 31, 'N': 64, 'Zn': 32}],
 'Al_Ge-H': [{'-1': -898.3837603, '0': -905.63633284, '1': -908.38812341},
  {'Al': 1, 'Ge': 31, 'H': 1, 'N': 64, 'Zn': 32}],
 'Al_Zn': [{'0': -904.62426901, '1': -911.39871314},
  {'Al': 1, 'Ge': 32, 'N': 64, 'Zn': 31}],
 'Al_i': [{'0': -897.93175756,
   '1': -904.84963109,
   '2': -911.21877123,
   '3': -917.78909189},
  {'Al': 1, 'Ge': 32, 'N': 64, 'Zn': 32}],
 'As_Ge': [{'-1': -891.8341766, '0': -898.52661309, '1': -905.2364975},
  {'As': 1, 'Ge': 31, 'N': 64, 'Zn': 32}],
 'As_N': [{'-1': -887.12366814, '0': -893.97325595, '1': -897.73490692},
  {'As': 1, 'Ge': 32, 'N': 63, 'Zn': 32}],
 'As_Zn': [{'-1': -894.67145705,
   '0': -900.64649732,
   '1': -906.80433712,
   '2': -911.64920084,
   '3': -916.98683613},
  {'As': 1, 'Ge': 32, 'N': 64, 'Zn': 31}],
 'Be_Ge': [{'-1': -893.70385805,
   '-2'

In [10]:
ChemicalPotentialReferences

{'Al': [-16.41683292, {'Al': 4}],
 'AlN': [-34.96263378, {'Al': 2, 'N': 2}],
 'As': [-33.75142116, {'As': 6}],
 'C': [-84.37207806, {'C': 8}],
 'Ca': [-3.61254745, {'Ca': 2}],
 'CaN': [-477.91231295, {'Ca': 48, 'N': 32}],
 'Cu': [-14.56107699, {'Cu': 4}],
 'Ga': [-27.06451165, {'Ga': 8}],
 'GaN': [-29.6145308, {'Ga': 2, 'N': 2}],
 'Ge': [-10.70087357, {'Ge': 2}],
 'Ge3N4': [-230.84178365, {'Ge': 12, 'N': 16}],
 'GeO2': [-72.90652552, {'Ge': 3, 'O': 6}],
 'GeS2': [-271.8334294, {'Ge': 16, 'S': 32}],
 'H': [-7.82474526, {'H': 2}],
 'In': [-11.69994163, {'In': 4}],
 'InN': [-25.98035876, {'In': 2, 'N': 2}],
 'Li': [-3.89574138, {'Li': 2}],
 'Li3N': [-17.72148574, {'Li': 3, 'N': 1}],
 'LiN3': [-66.13528866, {'Li': 2, 'N': 6}],
 'Mg': [-3.08794758, {'Mg': 2}],
 'N': [-20.44693316, {'N': 2}],
 'O': [-14.02604683, {'O': 2}],
 'P': [-51.69489998, {'P': 8}],
 'P3N5': [-297.47323234, {'N': 20, 'P': 12}],
 'S': [-41.96811057, {'S': 8}],
 'Se': [-13.35088177, {'Se': 3}],
 'Sn': [-36.04143642, {'Sn

In [11]:
-25.98035876 / 2.0 - -11.69994163 / 4 - -20.44693316/2

0.15827260749999894

In [4]:
Sc = -.15018323*100 /2

In [5]:
rs = -.21949453 * 100

In [6]:
zb = -.21256993*100

In [7]:
wu = -0.432863984870*100/2

In [8]:
N = -20.44693316/2

In [9]:
wu - Sc -N

-3.9105711634999984

In [10]:
zb - Sc - N

-3.5243649199999965

In [11]:
rs - Sc - N

-4.216824919999997

In [14]:
DefectTotalEnergiesScN

{'H_i': [{'-1': -693.16900562, '0': -700.365323, '1': -707.61446714},
  {'H': 1, 'N': 32, 'Sc': 32}],
 'default': [{'0': -697.91974502}, {'N': 32, 'Sc': 32}]}